<a href="https://colab.research.google.com/github/pearl-yu/mist5400fall2025/blob/main/3_GenAI/hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIST 5400 Lab: Using models from huggingFace

Adapted from huggingface example notebooks.

By Pearl Yu

# Quickstart

Transformers is designed to be fast and easy to use so that everyone can start learning or building with transformer models.

This quickstart introduces you to Transformers' key features and shows you how to:

- load a pre-trained model and run inference with [Pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.Pipeline)
- fine-tune a model with [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)

# First, install the transformers libraries

In [ ]:
# Transformers installation
! pip install transformers datasets evaluate accelerate

# Using Pretrained Models

The [Pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.Pipeline) class is the most convenient way to inference with a pretrained model. It supports many tasks such as text generation, image segmentation, automatic speech recognition, document question answering, and more.

> [!TIP]
> Refer to the [Pipeline](https://huggingface.co/docs/transformers/main/en/./main_classes/pipelines) API reference for a complete list of available tasks.

Create a [Pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.Pipeline) object and select a task. By default, [Pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.Pipeline) downloads and caches a default pretrained model for a given task. Pass the model name to the `model` parameter to choose a specific model.

<hfoptions id="pipeline-tasks">
<hfoption id="text generation">

Use `Accelerator` to automatically detect an available accelerator for inference.

## Text Generation

In [ ]:
from transformers import pipeline
from accelerate import Accelerator

device = Accelerator().device

pipeline_text = pipeline("text-generation", model="distilgpt2", device=device)

Prompt [Pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.Pipeline) with some initial text to generate more text.

In [ ]:
pipeline_text("The secret to baking a good cake is ", truncation=True)